# INCOMPLETE attempt to try to extract data from API... could not make sense of the credentials set up... Guides included steps that no longer exist in GA

In [ ]:
# Merge GA, impressions & product version data

To create features to try to explain 

In [1]:
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 6.5 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 861 kB/s eta 0:00:011
     |████████████████████████████████| 136 kB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 2.7 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.8 MB/s eta 0:00:011


In [3]:
!pip install --upgrade oauth2client

     |████████████████████████████████| 98 kB 3.6 MB/s eta 0:00:01


In [8]:
"""Hello Analytics Reporting API V4."""

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [ ]:
import json

cs = {
  "installed": {
    "client_id": "aj-session-location@analog-grin-308311.iam.gserviceaccount.com",
    "client_secret":"6558cf57b4b975b67fefd80789f2cf5b27db99b5",
    "redirect_uris": ["http://localhost", "urn:ietf:wg:oauth:2.0:oob"],
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://accounts.google.com/o/oauth2/token"
  }
}

# Serializing json  
json_object = json.dumps(cs, indent = 2) 
  
# Writing to sample.json 
with open("../src/client_secrets.json", "w") as outfile: 
    outfile.write(json_object) 


In [18]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
CLIENT_SECRETS_PATH = '../src/client_secrets.json'
VIEW_ID = '62984132'

In [19]:
def initialize_analyticsreporting():
    """Initializes the analyticsreporting service object.

    Returns:
    analytics an authorized analyticsreporting service object.
    """
    # Parse command-line arguments.
    parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
    flags = parser.parse_args([])

    # Set up a Flow object to be used if we need to authenticate.
    flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

    # Prepare credentials, and authorize HTTP object with them.
    # If the credentials don't exist or are invalid run through the native client
    # flow. The Storage object will ensure that if successful the good
    # credentials will get written back to a file.
    storage = file.Storage('analyticsreporting.dat')
    credentials = storage.get()
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    http = credentials.authorize(http=httplib2.Http())

    # Build the service object.
    analytics = build('analyticsreporting', 'v4', http=http)

    return analytics

def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}]
        }]
      }
  ).execute()

def print_response(response):
    """Parses and prints the Analytics Reporting API V4 response"""

    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

        for row in rows:
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                print(header + ': ' + dimension)

            for i, values in enumerate(dateRangeValues):
                print('Date range (' + str(i) + ')')
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                      print(metricHeader.get('name') + ': ' + value)


In [20]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)
print_response(response)

/opt/conda/lib/python3.8/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access analyticsreporting.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=aj-session-location%40analog-grin-308311.iam.gserviceaccount.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver



KeyboardInterrupt: 